In [1]:
from numpy import unique
from pandas import read_csv
url = 'https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv'
df = read_csv(url)
df.head()
df.count()

category    2225
text        2225
dtype: int64

In [2]:
#get the first 10 columns
df_len_10 = df['text'].iloc[:10]


In [14]:
#CLASSES
import random, math
import io
from itertools import chain
import pandas as pd

#QUESTION 1
class Shingling:
   
    def __init__(self, input_text, input_k):
        self.text = input_text
        self.k = input_k
           

    def shingles(self):
        for line in self.text:
            return {line[i:i + self.k] for i in range(len(line) - self.k + 1)} 
    
    
    def ordered_hash(self, input_sh):
        a = random.randint(0,100)
        b = random.randint(0,100)        #convert string into int
        return sorted(set((7*(int.from_bytes(x.encode(), 'little')) + 
                           13)%1039205197 for x in input_sh))
    
#QUESTION 2  
'''                
class CompareSets:
   
    @staticmethod
    def Jaccard(set1, set2):
        #return len(set1)/len(set2)
        return len(set1.intersection(set2)) / len(set1.union(set2))
'''    
class CompareSetsInput:
    def __init__(self, set1, set2):
        self.seta = set1
        self.setb = set2
    
    def Jac_Sim(self):
        return len(self.seta.intersection(self.setb)) / len(self.seta.union(self.setb))
    

#Question 3
class MinHashing:

    def __init__(self, input_set, input_n):
        self.s = input_set
        self.n = input_n

    def hashing(self):
        min_hash = [None] * self.n # Vector of length n
        random.seed(111)
        for n in range(self.n):
            a = random.randint(0, 2**32-1) # Max shingle id
            b = random.randint(0, 2**32-1)
            vector = [None] * len(self.s) # Vector of length set hashed shingles
            for i, value in enumerate(self.s):
                hashcode = ((a * value + b) % 4294967311) # Make sure that prime > max shingle id
                vector[i] = hashcode 
            min_hash[n] = min(vector)
        return min_hash
    
#Question 4    
class CompareSignatures:
    def __init__(self, vector1, vector2):
        self.v1 = vector1
        self.v2 = vector2

    def estimate(self):
        count = 0
        for i, elem in enumerate(self.v1):
            if self.v1[i] == self.v2[i]: # Assumed that length of both vectors are the same
                count += 1

        return count/len(self.v1)
    
#Qustion 5 

class LHS:
    def __init__(self, signatures, n, threshold):
        self.signatures = signatures
        self.n = n
        self.threshold = threshold
    
     
    def candidates(self):
        b, total_rows, total_docs = self.n, self.signatures.count().min(), len(self.signatures.columns)
        r = math.floor(total_rows / b)
        
        bucket = {} #dictionary
        cand = []
        
        for i in range(b):
            band = self.signatures[r*i:r*(i+1)]   #bands
            
            for j in range(total_docs):
                bucket[j] = hash(tuple(band[j].values)) #hash content of bands-->same bands will have same hash value
           
                
            for f in range(total_docs):
                for s in range(f+1, total_docs):
                    if bucket[f]==bucket[s]:
                        cand.append([f,s])    #get pairs of columns with same hash value (identical bands)
            
        self.final_cand = [list(x) for x in set(tuple(x) for x in cand)]      
        return(self.final_cand)
    
    def check_similarity(self):
        if self.final_cand:
            rows = self.signatures.count().min()
            for pair in self.final_cand:
                col1 = pair[0]
                col2 = pair[1]
                mh = []
                for i in [col1,col2]:
                    ordered_set2 = set(self.signatures[i].iloc[:rows])
                    mh.append(MinHashing(ordered_set2,20).hashing())

                if CompareSignatures(mh[0],mh[1]).estimate() <= self.threshold:
                    print("Column",col1,"and",col2,"have Jaccard similarity:",
                         CompareSignatures(mh[0],mh[1]).estimate(),"which is lower than",self.threshold,
                          "so they are not similar")
                else:        
                    print("Column",col1,"and",col2,"have Jaccard similarity:",
                          CompareSignatures(mh[0],mh[1]).estimate(),"which is higher than",self.threshold,"so they are similar")
        else:
            print("No candidates")

In [15]:
#get shingles and compute Jaccard similarity
text1 = io.StringIO(df_len_10[0])
text2 = io.StringIO(df_len_10[1])
sh1 = Shingling(text1,10)
shingles1 = sh1.shingles()
ordered = sh1.ordered_hash(shingles1)
print(len(ordered),"ordered=",ordered)
sh2 = Shingling(text2,10)
shingles2 = sh2.shingles()
Jac = CompareSetsInput(shingles1, shingles2)
Sim = Jac.Jac_Sim()
print(Sim)


4114 ordered= {663494657, 852213763, 9863173, 103956485, 127467526, 147136517, 421003269, 682385415, 804347909, 842334219, 877494277, 458620942, 897343496, 230162448, 763232272, 1029259280, 754974739, 182796310, 72933402, 513335322, 338042909, 574341150, 590430239, 765411359, 494215204, 896131108, 479903782, 401432615, 509984808, 411369513, 123494442, 335642667, 697819179, 838975530, 988831795, 88866873, 341401658, 447496251, 657760313, 753524796, 51281985, 43606087, 47841351, 180568136, 610541640, 647749708, 70320205, 347578445, 1008713807, 679903312, 420216913, 900857939, 750723163, 13377631, 845922404, 417783911, 1024073832, 508633193, 580853867, 974946412, 565215342, 387301487, 430669936, 334299250, 434372723, 290365556, 858710134, 978616440, 454672505, 1001537657, 258228347, 979632252, 915169409, 867426434, 22110341, 109437062, 254632069, 421789829, 537485445, 426303626, 677240966, 1000808585, 1024745607, 1030643854, 132079760, 676307090, 738386072, 840786075, 47841442, 473604258,

In [8]:
#q3
sample_text = io.StringIO(df_len_10[0])
# Construct 10-shingles
sh = Shingling(sample_text,10) 
shingles = sh.shingles() 
# Ordered set of hashed 100-shingles
ordered_set = sh.ordered_hash(shingles) 


mh = MinHashing(ordered_set,20)
minhash = mh.hashing()
print(minhash)

[1397187, 1720018, 2280780, 2509501, 447296, 2695677, 398046, 2199526, 3766391, 85862, 713034, 434350, 405552, 517131, 746466, 1313464, 40723, 1233335, 1183766, 2193635]


In [9]:
#q4
sets = []
for i in range(2):
    sample_text = io.StringIO(df_len_10[i])
    # Construct 10-shingles
    sh = Shingling(sample_text,10) 
    shingles = sh.shingles() 
    # Ordered set of hashed 100-shingles
    ordered_set = sh.ordered_hash(shingles)
    sets.append(ordered_set)


mh1 = MinHashing(sets[0],20).hashing()
mh2 = MinHashing(sets[1],20).hashing()
print(CompareSignatures(mh1,mh1).estimate())
print(CompareSignatures(mh1,mh2).estimate())

1.0
0.0


In [10]:
#input signatures in LHS
df = pd.DataFrame(sets).transpose()
df.head()

,0,1
0,40577.0,37520.0
1,551102.0,41153.0
2,580363.0,982770.0
3,645795.0,1026742.0
4,765198.0,1066915.0


In [11]:
#q5 candidates and their similarity
l = LHS(df, 200, 0.8)
l.candidates()
l.check_similarity()

No candidates


In [12]:
import copy
df_with_duplicates = copy.deepcopy(df) 
df_with_duplicates[2] = df_with_duplicates[0]
df_with_duplicates[3] = df_with_duplicates[1]
print(df_with_duplicates.head())
print(df.head())

l_test = LHS(df_with_duplicates, 200, 0.8)
l_test.candidates()
l_test.check_similarity()

          0          1         2          3
0   40577.0    37520.0   40577.0    37520.0
1  551102.0    41153.0  551102.0    41153.0
2  580363.0   982770.0  580363.0   982770.0
3  645795.0  1026742.0  645795.0  1026742.0
4  765198.0  1066915.0  765198.0  1066915.0
          0          1
0   40577.0    37520.0
1  551102.0    41153.0
2  580363.0   982770.0
3  645795.0  1026742.0
4  765198.0  1066915.0
Column 0 and 2 have Jaccard similarity: 1.0 which is higher than 0.8 so they are similar
Column 1 and 3 have Jaccard similarity: 1.0 which is higher than 0.8 so they are similar
